In [3]:
import lightgbm as lgb
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# 1. Cargar dataset
df = pd.read_csv(r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\datasetwg_features.csv")

# 2. Renombrar columnas para evitar caracteres especiales
df.columns = [col.replace("(", "_").replace(")", "").replace(",", "_").replace(" ", "_") for col in df.columns]

# 3. Label Encoding para columnas categóricas
for col in df.select_dtypes(include="object").columns:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# 4. Separar X e y
X = df.drop(columns=["tn"])
y = df["tn"]

# 5. Split train / valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Crear datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

# 7. Definir función objetivo
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "feature_pre_filter": False,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0)
    }

    model = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(50),
            lgb.log_evaluation(100)
        ]
    )

    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    return rmse

# 8. Crear y ejecutar estudio Optuna
storage = "sqlite:///optuna_lgbm_study.db"
study = optuna.create_study(direction="minimize", study_name="optuna_tn_studyw7", storage=storage, load_if_exists=True)
study.optimize(objective, n_trials=50)

# 9. Mostrar mejores hiperparámetros
print("\n🎯 Mejores hiperparámetros encontrados:")
print(study.best_params)


[I 2025-06-24 21:48:27,483] A new study created in RDB with name: optuna_tn_studyw7


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.622277
Early stopping, best iteration is:
[90]	valid_0's rmse: 0.621323


[I 2025-06-24 21:49:45,690] Trial 0 finished with value: 0.6213231727842913 and parameters: {'learning_rate': 0.22964921091172663, 'num_leaves': 178, 'min_data_in_leaf': 15, 'lambda_l1': 1.6829328757591355, 'lambda_l2': 0.13618439852783715}. Best is trial 0 with value: 0.6213231727842913.


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.604834
Early stopping, best iteration is:
[50]	valid_0's rmse: 0.602391


[I 2025-06-24 21:50:55,673] Trial 1 finished with value: 0.6023906997685285 and parameters: {'learning_rate': 0.1750995286210126, 'num_leaves': 254, 'min_data_in_leaf': 26, 'lambda_l1': 3.184853212835522, 'lambda_l2': 2.237942453026239}. Best is trial 1 with value: 0.6023906997685285.


...

Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.62478
Early stopping, best iteration is:
[51]	valid_0's rmse: 0.620968


[I 2025-06-24 23:37:00,376] Trial 48 finished with value: 0.620967646388639 and parameters: {'learning_rate': 0.2990312423199424, 'num_leaves': 158, 'min_data_in_leaf': 52, 'lambda_l1': 2.051973478169594, 'lambda_l2': 4.110829385767673}. Best is trial 42 with value: 0.5902676727099929.


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.637381
[200]	valid_0's rmse: 0.602163
[300]	valid_0's rmse: 0.597431
[400]	valid_0's rmse: 0.59692
Early stopping, best iteration is:
[367]	valid_0's rmse: 0.596738


[I 2025-06-24 23:38:48,348] Trial 49 finished with value: 0.5967382271785875 and parameters: {'learning_rate': 0.03358090448054765, 'num_leaves': 80, 'min_data_in_leaf': 13, 'lambda_l1': 2.4116917607888793, 'lambda_l2': 4.859607837019257}. Best is trial 42 with value: 0.5902676727099929.



🎯 Mejores hiperparámetros encontrados:
{'learning_rate': 0.04139511287713252, 'num_leaves': 197, 'min_data_in_leaf': 25, 'lambda_l1': 2.1394729782629565, 'lambda_l2': 4.709963551310892}
